# Mushroom Detection Model - Test and Implementation

## Import Basic Libraries

In [3]:
# import default libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil
import torch

In [1]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.17  Python-3.12.6 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
Setup complete  (16 CPUs, 31.7 GB RAM, 578.5/931.0 GB disk)


In [18]:
test_image_dir = 'mushroom-data/test/images/*'
test_label_dir = 'mushroom-data/test/labels/*'

In [42]:
mush_class_dict = {'0': 'Amanita citrina', 
              '1': 'Amanita hemibapha', 
              '2': 'Amanita javanica', 
              '3': 'Amanita pantherina', 
              '4': 'Amanita princeps', 
              '5': 'Boletus reticulatus', 
              '6':'Coltricia perennis', 
              '7': 'Gyromitra infula', 
              '8':'Hygrophoropsis aurantiaca', 
              '9': 'Imleria badia', 
              '10': 'Lactarius turpis', 
              '11': 'Russula emetica', 
              '12': 'Suillus grevillei', 
              '13': 'Suillus luteus'}

In [45]:
def get_ground_truth(image_path, label_dir, class_dict):
    root_name = os.path.basename(image_path).removesuffix('.jpg')
    mush_instances = []
    mush_instance_names = []
    for item in glob.glob(label_dir):
        txt_file = os.path.basename(item)
        if root_name in txt_file:
            file_location = label_dir.strip('*') + '/' + txt_file
            file_data = open(file_location, "r")
            for line in file_data:
                line.split()
                data = line.split(' ')
                mush_class = data[0]
                mush_instances.append(mush_class)
    for mush in mush_instances:
        instance_class = class_dict[mush]
        mush_instance_names.append(instance_class)
    return mush_instance_names
        

In [46]:
get_ground_truth("mushroom-data/test/images/6652medium_jpg.rf.d2aae7353fc0d694f8215c7b4714ea5f.jpg", test_label_dir, mush_class_dict)

['Coltricia perennis', 'Coltricia perennis', 'Coltricia perennis']